In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('./data/rev_ref.csv', encoding='utf-8')

In [5]:
df = df.set_index('nid')

In [6]:
cite_dict = df.to_dict('index')

In [58]:
for key in cite_dict:
    print(key, cite_dict[key])
    break

0 {'id': '000000b8-7f59-49ad-b9bc-e92aa858fc37', 'cited': '[877113, 1738015, 1801628, 2375682, 2398534, 2574460, 3122433, 3392608, 3545645, 3593462, 4325752, 4787741, 4916987, 5489857, 5556256, 5615564, 6636660, 6729355, 6729360, 6887341, 6920577, 7009207, 7409995, 7617229, 7700168, 7730619, 7824727, 7825621, 7838344, 8227147, 8473347, 8631855, 9141446, 9327239, 9623822, 9761228, 10039120, 10540891, 10617720, 10866595, 11110006, 11227418, 11535220, 11693317]', 'n_ref': 17, 'rank_score': 1.9485039745235887e-08}


with open('./data/rev_ref.json', 'r') as f:
    cite_dict = json.load(f, encoding='utf-8')

for idx, iid in enumerate(cite_dict):
    if type(cite_dict[iid]['cited']) == str:
        cite_dict[iid]['cited'] = json.loads(cite_dict[iid]['cited'])

with open('./data/rev_ref.json', 'w') as f:
    json.dump(cite_dict, f, ensure_ascii=False)

In [4]:
with open('./data/paper_author_coef.json', 'r') as f:
    paper_author_coef = json.load(f, encoding='utf-8')

In [53]:
total_count = len(cite_dict)
for key in cite_dict:
    cite_dict[key]['rank_score'] = 1/float(total_count)

In [54]:
len(cite_dict)

11763764

In [55]:
def paper_rank2(dic):
    total_count = len(dic)

    for _ in range(3):
        for idx, p in enumerate(dic):
            print(idx) if idx % 1000000 == 0 else None
            rank_sum = 0
#             print(dic[p])
            cited_list = json.loads(dic[p]['cited'])
            if cited_list:
#                 print(cited_list)
                for c in cited_list:
    #                 print(c)
                    c_row = dic[c]
                    cite_cnt = c_row['n_ref']
                    rank_sum += (1/float(cite_cnt)) * c_row['rank_score']
            dic[p]['rank_score'] = 0.45*(1/float(total_count)/float(paper_author_coef[p])) + 0.55*rank_sum
#             print(dic[p]['rank_score'])

In [56]:
paper_rank2(cite_dict)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000


In [57]:
with open('./data/pagerank_3_2.json', 'w') as f:
    json.dump(cite_dict, f, ensure_ascii=False)

In [61]:
for idx, author in enumerate(author_article):
    for iid in author_article[author]['id']:
        author_article[author]['rank_score'].append(cite_dict[iid]['rank_score'])

In [62]:
next(iter(author_article.values()))

{'id': [0, 1013803, 8227147, 8634565, 9327239],
 'order': [0, 0, 0, 0, 0],
 'rank_score': [1.9485039745235887e-08,
  8.819456085654217e-09,
  1.0982444346936832e-08,
  9.419237027924808e-09,
  8.012927361414515e-09]}

In [63]:
with open('./data/author_score_3.json', 'w') as f:
    json.dump(author_article, f, ensure_ascii=False)

In [ ]:
cite_score = pd.DataFrame.from_dict(cite_dict, orient='index')
cite_score.rename(columns=dict(zip(cite_score.columns[[0,1,2]], ['cited', 'n_ref', 'rank_score'])),inplace=True)
cite_score.head()

### id-author coef

In [7]:
with open('./data/mag_list_noid.json', 'r') as f:
    mag_list = json.load(f, encoding='utf-8')

In [50]:
author_article = {}
for idx, row in enumerate(mag_list):
    al = json.loads(row['authors'])
    for order, author in enumerate(al):
        if author not in author_article:
            author_article[author] = {'id': [], 'order': [], 'rank_score': []}
        author_article[author]['id'].append(idx)
        author_article[author]['order'].append(order)

In [52]:
len(author_article)

6872744

In [60]:
next(iter(author_article.values()))

{'id': [0, 1013803, 8227147, 8634565, 9327239],
 'order': [0, 0, 0, 0, 0],
 'rank_score': []}

In [51]:
with open('./data/author_article.json', 'w') as f:
    json.dump(author_article, f, ensure_ascii=False)

In [23]:
paper_author_coef = []
for idx, row in enumerate(mag_list):
    al = json.loads(row['authors'])
    coef = 0
    for order, author in enumerate(al):
#         print(len(author_article[author]['id']))
        coef += len(author_article[author]['id']) * (2**-order)
    paper_author_coef.append(coef)

In [24]:
len(paper_author_coef)

11763764

In [27]:
paper_author_coef[0]

8.25

In [26]:
with open('./data/paper_author_coef.json', 'w') as f:
    json.dump(paper_author_coef, f, ensure_ascii=False)